# Notebook 5 : Réduction de dimension

Notebook préparé par [Chloé-Agathe Azencott](http://cazencott.info).

Dans ce notebook il s'agit d'explorer plusieurs techniques de réduction de dimension.

In [ ]:
# charger numpy as np, matplotlib as plt
%pylab inline 

In [ ]:
plt.rc('font', **{'size': 12}) # règle la taille de police globalement pour les plots (en pt)

In [ ]:
import pandas as pd

## 1. Analyse en composantes principales

Dans cette section, nous allons effectuer une analyse en composantes principales d'un jeu de données décrivant les scores obtenus par les meilleurs athlètes ayant participé en 2004 à une épreuve de décathlon, aux Jeux Olympiques d'Athènes ou au Décastar de Talence.

### Chargement des données

Les données sont contenues dans le fichier `decathlon.txt`.

Le fichier contient 42 lignes et 13 colonnes.

La première ligne est un en-tête qui décrit les contenus des colonnes.

Les lignes suivantes décrivent les 41 athlètes.

Les 10 premières colonnes contiennent les scores obtenus aux différentes épreuves.
La 11ème colonne contient le classement.
La 12ème colonne contient le nombre de points obtenus.
La 13ème colonne contient une variable qualitative qui précise l'épreuve (JO ou Décastar) concernée.

Nous allons examiner ces données avec la librairie `pandas`.

In [ ]:
my_data = pd.read_csv('data/decathlon.txt', sep="\t")  # lire les données dans un dataframe

__Alternativement :__ Si vous avez besoin de télécharger le fichier (par exemple sur colab) :

In [ ]:
!wget https://raw.githubusercontent.com/chagaz/cp-ia-intro-ml-2022/main/5-Reduction/data/decathlon.txt

my_data = pd.read_csv('decathlon.txt', sep="\t") 

In [ ]:
my_data.head()

### Visualisation

Une __matrice de nuages de points__ est une visualisation en k x k panneaux des relations deux à deux entre k variables :
* sur la diagonale, l'histogramme pour chacune des variables 
* hors diagonale, les nuages de points entre deux variables (non standardisées).

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.plotting.scatter_matrix.html

Par exemple :

In [ ]:
from pandas.plotting import scatter_matrix

scatter_matrix(my_data[['Shot.put','High.jump', '400m']], alpha=0.5, s=60,
               figsize=(8, 8));

In [ ]:
from pandas.plotting import scatter_matrix

scatter_matrix(my_data, alpha=0.5, s=60,
               figsize=(18, 18));

Alternativement, la librairie `seaborn` permet des visualisations plus élaborées que `matplotlib`. Vous pouvez par exemple explorer les capacités de `jointplot`. 
https://seaborn.pydata.org/generated/seaborn.jointplot.html

In [ ]:
import seaborn as sns
sns.set_style('whitegrid')

sns.jointplot(x='Shot.put', y='400m', data = my_data, 
              height=6, space=0, color='b')

sns.jointplot(x='Shot.put', y='400m', data = my_data, 
              kind='reg', height=6, space=0, color='b')

Nous allons maintenant effectuer une analyse en composantes principales des scores aux 10 épreuves.

Commençons par extraire les données :

In [ ]:
X = np.array(my_data.drop(columns=['Points', 'Rank', 'Competition']))
print(X.shape)

### Standardisation des données

In [ ]:
from sklearn import preprocessing

In [ ]:
std_scale = preprocessing.StandardScaler().fit(X)
X_scaled = std_scale.transform(X)

### Calcul des composantes principales

Les algorithmes de factorisation de matrice de `scikit-learn` sont inclus dans le module `decomposition`. Pour  l'ACP, référez-vous à : 
http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [ ]:
from sklearn import decomposition

Remarque : Nous avons ici peu de variables et pouvons nous permettre de calculer toutes les PC. 

La plupart des algorithmes implémentés dans `scikit-learn` suivent le fonctionnement suivant : 
* on instancie un objet, correspondant à un type d'algorithme/modèle, avec ses hyperparamètres (ici le nombre de composantes)
* on utilise la méthode `fit` pour passer les données à cet algorithme
* les paramètres appris sont maintenant accessibles comme arguments de cet objet.

In [ ]:
# Instanciation d'un objet PCA pour 10 composantes principales
pca = decomposition.PCA(n_components=10)

In [ ]:
# On passe maintenant les données standardisées à cet objet
# C'est ici que se font les calculs
pca.fit(X_scaled)

### Proportion de variance expliquée par les PCs

Nous allons maintenant afficher la proportion de variance expliquée par les différentes composantes. Il est accessible dans le paramètre `explained_variance_ration_` de notre objet `pca`.

In [ ]:
plt.plot(np.arange(1, 11), pca.explained_variance_ratio_, marker='o')

plt.xlabel("Nombre de composantes principales")
plt.ylabel("Proportion de variance expliquée")

Nous pouvons aussi afficher la proportion *cumulative* de variance expliquée

In [ ]:
np.cumsum(pca.explained_variance_ratio_)

In [ ]:
plt.plot(np.arange(1, 11), np.cumsum(pca.explained_variance_ratio_), marker='o')

plt.xlabel("Nombre de composantes principales")
plt.title("Proportion cumulative de variance expliquée")

On peut aussi calculer la proportion de variance expliquée par les 4 (par exemple) premières PC avec

In [ ]:
print("%.2f" % np.sum(pca.explained_variance_ratio_[:4]))

__Questions :__ 
* Quelle est la proportion de variance expliquée par les deux premières composantes ? 
* Combien de composantes faudrait-il utiliser pour expliquer 80% de la variance des données ?

In [ ]:
print("%.2f" % np.sum(pca.explained_variance_ratio_[:2]))

### Projection des données sur les deux premières composantes principales

Nous allons maintenant utiliser uniquement les deux premières composantes principales.

Commençons par calculer la nouvelle représentation des données, c'est-à-dire leur projection sur ces deux PC. 

In [ ]:
pca = decomposition.PCA(n_components=2)
pca.fit(X_scaled)
X_projected = pca.transform(X_scaled)
print(X_projected.shape)

On peut afficher un nuage de points représentant les données selon ces deux PC.

In [ ]:
fig = plt.figure(figsize=(5, 5))

plt.scatter(X_projected[:, 0], X_projected[:, 1])

plt.xlabel("PC 1")
plt.ylabel("PC 2")

On peut maintenant colorer chaque point du nuage de points ci-dessus en fonction du classement de l'athlète qu'il représente. 

In [ ]:
fig = plt.figure(figsize=(5, 5))

plt.scatter(X_projected[:, 0], X_projected[:, 1], c=my_data['Rank'])

plt.xlabel("PC 1")
plt.ylabel("PC 2")
plt.colorbar(label='classement')

__Question :__ Qu'en conclure sur l'interprétation de la PC1 ?

### Interprétation des deux composantes principales
Chaque composante principale est une combinaison linéaire des variables décrivant les données. Les poids de cette combinaison linéaire sont accesibles dans `pca.components_`.

Nous pouvons visualiser non pas les individus, mais les 10 variables dans l'espace des 2 composantes principales.

In [ ]:
pcs = pca.components_
print(pcs[0])

In [ ]:
fig = plt.figure(figsize=(6, 6))

plt.scatter(pcs[0], pcs[1])
for (x_coordinate, y_coordinate, feature_name) in zip(pcs[0], pcs[1], my_data.columns[:10]):
    plt.text(x_coordinate, y_coordinate, feature_name)                          
    
plt.xlabel("Contribution à la PC1")
plt.ylabel("Contribution à la PC2")

__Question :__ Quelles variables ont des contributions très similaires aux deux composantes principales ? Qu'en déduire sur leur similarité ?

__Question :__ Comment interpréter le signe des contributions des variables à la première composantes principales ?

## 2. Données « Olivetti »

Nous allons maintenant utiliser la réduction de dimension pour représenter en deux dimensions un jeu de données contenant des visages. Il s'agit d'un jeu de données classique, contenant 400 photos de 64 par 64 pixels. Il s'agit de photos des visages de 40 personnes différentes (10 photos par personne), étiquetées par un numéro de classe entre 0 et 39 identifiant la personne.

Nous pouvons charger ce jeu de données directement grâce à scikit-learn :

In [ ]:
from sklearn import datasets

In [ ]:
data = datasets.fetch_olivetti_faces()

__Si vous n'arrivez pas à télécharger les données :__
* Aller sur : https://github.com/CroncLee/PCA-face-recognition/blob/master/olivetti_py3.pkz
* Télécharger le fichier (bouton Download) 
utiliser la commande
```
    data = datasets.fetch_olivetti_faces(data_home="<PATH TO DATA>")
```
En remplaçant <PATH TO DATA> par le chemin vers le dossier où vous avez enregistré les données.

In [ ]:
X = data.data
y = data.target

In [ ]:
print(X.shape)

In [ ]:
print("Les données contiennent %d classes" % len(np.unique(y)))

Chaque image est représentée par une valeur (niveau de gris) pour chacun de ses pixels. 

Nous pouvons visualiser ces images à condition de réorganiser ces valeurs (= un vecteur de longueur 4096) en matrices 64x64. Par exemle ci-dessous pour l'image à l'index 23 :

In [ ]:
plt.imshow(X[13, :].reshape((64, 64)), interpolation='nearest', cmap=plt.cm.gray)

### PCA

Commençons par une analyse en composantes principales comme à la section précédente :

In [ ]:
pca = decomposition.PCA(n_components=2)
X_transformed_pca = pca.fit_transform(X)

Chaque image est maintenant représentée par non pas 4096 variables, mais par deux variables. Nous pouvons les visualiser en nuage de point, et les colorer par classe :

In [ ]:
plt.scatter(X_transformed_pca[:, 0], X_transformed_pca[:, 1], 
            c=y)
plt.xlabel("Première composante principale")
plt.ylabel("Deuxième composante principale")

__Question :__ les images du même visage (= de la même classe) ont-elles des représentations proches ?

Nous pouvons visualiser la contribution de chaque pixel à la première composante principale :

In [ ]:
plt.imshow(pca.components_[0, :].reshape((64,64)), interpolation='nearest', cmap=plt.cm.gray)

Puis la contribution de chaque pixel à la deuxième composante principale :

In [ ]:
plt.imshow(pca.components_[1, :].reshape((64,64)), interpolation='nearest', cmap=plt.cm.gray)

__Question :__ 
* Quelle est la proportion de variance expliquée par les deux premières composantes ? 
* En observant les 2 premières composantes principales, que remarquez-vous sur les zones ou les structures mises en avant dans l'image ? Que peut-on en déduire sur les informations qu'elles capturent ?

### tSNE

Nous allons maintenant utiliser la même démarche, mais avec tSNE, grâce à la classe [TSNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) du module `manifold`.

In [ ]:
from sklearn import manifold

In [ ]:
tsne = manifold.TSNE(n_components=2, init='random', learning_rate='auto')

In [ ]:
%%time 
X_transformed = tsne.fit_transform(X)

In [ ]:
plt.scatter(X_transformed[:, 0], X_transformed[:, 1], 
            c=y)
plt.xlabel("Première composante tSNE")
plt.ylabel("Deuxième composante tSNE")

#### Influence du paramètre de perplexité 

In [ ]:
tsne = manifold.TSNE(n_components=2, init='random', learning_rate='auto', perplexity=1)

In [ ]:
%%time 
X_transformed = tsne.fit_transform(X)

In [ ]:
plt.scatter(X_transformed[:, 0], X_transformed[:, 1], 
            c=y)
plt.xlabel("Première composante tSNE")
plt.ylabel("Deuxième composante tSNE")

In [ ]:
tsne = manifold.TSNE(n_components=2, init='random', learning_rate='auto', perplexity=100)

In [ ]:
%%time 
X_transformed = tsne.fit_transform(X)

In [ ]:
plt.scatter(X_transformed[:, 0], X_transformed[:, 1], 
            c=y)
plt.xlabel("Première composante tSNE")
plt.ylabel("Deuxième composante tSNE")

__Question :__ Comment les résultats de la visualisation t-SNE changent-ils lorsque vous modifiez la perplexité ? Dans le premier graphique, que vaut le paramètre perplexity ?

### UMAP

Nous allons maintenant utiliser la même démarche, mais avec la bibliothéque [UMAP](https://umap-learn.readthedocs.io/en/latest/)

In [ ]:
pip install umap-learn

In [ ]:
import umap.umap_ as umap

In [ ]:
reducer = umap.UMAP()

In [ ]:
%%time 
X_umap = reducer.fit_transform(X)

In [ ]:
plt.scatter(X_umap[:, 0], X_umap[:, 1], 
            c=y,cmap = 'gist_rainbow')
plt.xlabel("Première composante Umap")
plt.ylabel("Deuxième composante Umap")

__Question :__ Aller voir dans la documentation [UMAP](https://umap-learn.readthedocs.io/en/latest/), les différents paramètres de la méthode UMAP. Lesquels sont choisis par défaut ? Les modifier pour tester leur impact.